<a href="https://colab.research.google.com/github/mario-rot/Introduction-to-Human-Language-Technology/blob/main/IHLTscratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell
git clone https://github.com/mario-rot/Introduction-to-Human-Language-Technology.git
cd 'Introduction-to-Human-Language-Technology'
mv 'Complementary Material' /content/
mv 'text_processing.py' /content/

pip install svgling

fatal: destination path 'Introduction-to-Human-Language-Technology' already exists and is not an empty directory.
mv: cannot stat 'Complementary Material': No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
import math
from nltk.util import bigrams, trigrams
from nltk.corpus import wordnet as wn
from text_processing import text_processing, compute_metrics
from collections import Counter
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
dt = pd.read_csv('/content/Complementary Material/test-gold/STS.input.SMTeuroparl.txt',sep='\t',header=None)

In [ ]:
dt.head()

,0,1
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi..."


In [ ]:
sentences1 = dt[0]
sentences2 = dt[1]

tp1 = text_processing(sentences1).lemmatize_data()
tp2 = text_processing(sentences2).lemmatize_data()


 -- Data hasn't been cleaned, to lemmatize the data is going to be cleaned with the default parameters --


 -- Data hasn't been cleaned, to lemmatize the data is going to be cleaned with the default parameters --



In [ ]:
cln1 = text_processing(sentences1).clean_data()
cln2 = text_processing(sentences2).clean_data()

mcl1 = text_processing(sentences1).mc_lemmatize_data(lemma=True)
mcl2 = text_processing(sentences2).mc_lemmatize_data(lemma=True)

syn1 = [[lemma.synset() for lemma in sentence if lemma] for sentence in mcl1]
syn2 = [[lemma.synset() for lemma in sentence if lemma] for sentence in mcl2]


 -- Data hasn't been cleaned, to apply most_common_lemma to data is going to be cleaned with the default parameters --


 -- Data hasn't been cleaned, to apply most_common_lemma to data is going to be cleaned with the default parameters --



In [ ]:
mcl1[95]

[Lemma('addition.n.03.increase'),
 Lemma('power.n.01.power'),
 Lemma('large.a.01.big'),
 Lemma('state.n.04.country'),
 Lemma('expense.v.01.expense'),
 Lemma('small.a.01.small'),
 Lemma('state.n.04.country')]

In [ ]:
def clean(corpus, stopwords, minwords_len, signs):
   corpus = corpus.split(' ')
   corpus = [word.lower() for word in corpus]
   corpus = [word if not any(caracter in signs for caracter in word) else remove_signs(word) for word in corpus]
   corpus = [word for word in corpus if word not in stopwords and word.isalpha()]
   corpus = [word for word in corpus if len(word) > minwords_len]
   return corpus

def remove_signs(wrd):
  wrd = list(wrd)
  wrd = [word for word in wrd if not any(caracter in signs for caracter in word)]
  wrd = ''.join(wrd)
  return wrd


stopwords=set(nltk.corpus.stopwords.words('english'))
signs = string.punctuation
minwords_len = 2

In [ ]:
def cosine_similarity(sentence1, sentence2):
  sim = 0
  all_words = set(sentence1).union(set(sentence2))
  s1vec = []
  s2vec = []
  for word in all_words:
    if word in sentence1:
      s1vec.append(1)
    else:
      s1vec.append(0)
    if word in sentence2:
      s2vec.append(1)
    else:
      s2vec.append(0)
  sim = np.dot(np.array(s1vec), np.array(s2vec)) / (math.sqrt(sum(s1vec)*sum(s2vec)))
  print(sentence1)
  print(sentence2)
  print(all_words)
  print(s1vec)
  print(s2vec)
  print(sim)
  return sim

In [ ]:
sent1 = clean(dt[0][0], stopwords, minwords_len, signs)
sent2 = clean(dt[1][0], stopwords, minwords_len, signs)

In [ ]:
cosine_similarity(sent1, sent2)

['leaders', 'given', 'new', 'chance', 'let', 'hope', 'seize']
['leaders', 'benefit', 'aujourd', 'hui', 'new', 'luck', 'lets', 'let', 'therefore', 'seize']
{'therefore', 'new', 'seize', 'given', 'luck', 'hope', 'leaders', 'lets', 'hui', 'benefit', 'let', 'chance', 'aujourd'}
[0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0]
[1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1]
0.47809144373375745


0.47809144373375745

In [ ]:
def unigram_similarity(words1, words2):
    count_same = 0
    for w in words1:
        count_same += min(words1.count(w), words2.count(w))
    
    if len(words1) > 0 or len(words2) > 0:
        return 2*count_same/(len(words1)+len(words2))
    else:
        return 0

In [ ]:
unigram_similarity(sent1, sent2)

0.47058823529411764

In [ ]:
8/17

0.47058823529411764

In [ ]:
def unigram_similarity_mine(sentence1, sentence2, remove_duplicates):
  if remove_duplicates:
    sentence1 = list(set(sentence1))
    sentence2 = list(set(sentence2))
    print(sentence1)
    print(sentence2)
  total_words = len(sentence1) + len(sentence2)
  used_words = []
  count = 0
  for word in sentence1:
    if word in sentence2 and word not in used_words:
      count += sentence1.count(word)
      count += sentence2.count(word)
    used_words.append(word)

  sim = count / total_words
  return sim

In [ ]:
unigram_similarity_mine(sent1, sent2, False)

0.5625

In [ ]:
dt[0][95]

'It increases the power of the big countries at the expense of the small countries.'

In [ ]:
sentence1 = 'It increases the power of the big countries at the expense of the small countries.'
sentence2 = 'This has the effect of increasing the power of large countries at the expense of smaller countries.'


sent1 = clean(sentence1, stopwords, minwords_len, signs)
sent2 = clean(sentence2, stopwords, minwords_len, signs)

print(unigram_similarity(sent1, sent2))
print(unigram_similarity_mine(sent1, sent2, False))
print(sent1)
print(sent2)
print(unigram_similarity_mine(sent1, sent2, True))

0.8
0.5333333333333333
['increases', 'power', 'big', 'countries', 'expense', 'small', 'countries']
['effect', 'increasing', 'power', 'large', 'countries', 'expense', 'smaller', 'countries']
['countries', 'power', 'expense', 'increases', 'big', 'small']
['effect', 'countries', 'power', 'increasing', 'smaller', 'expense', 'large']
0.46153846153846156


In [ ]:
print(unigram_similarity_mine(tp1[95], tp2[95], True))
print(unigram_similarity_mine(tp1[95], tp2[95], False))

['power', 'increase', 'expense', 'big', 'country', 'small']
['effect', 'power', 'increase', 'expense', 'large', 'country', 'small']
0.7692307692307693
0.7857142857142857


In [ ]:
print(mcl1[95])
print(mcl2[95])

[Lemma('addition.n.03.increase'), Lemma('power.n.01.power'), Lemma('large.a.01.big'), Lemma('state.n.04.country'), Lemma('expense.v.01.expense'), Lemma('small.a.01.small'), Lemma('state.n.04.country')]
[Lemma('consequence.n.01.effect'), Lemma('increase.v.01.increase'), Lemma('power.n.01.power'), Lemma('large.a.01.large'), Lemma('state.n.04.country'), Lemma('expense.v.01.expense'), Lemma('small.a.01.small')]


In [ ]:
mcl195 = [lemma for lemma in mcl1[0] if lemma]
mcl295 = [lemma for lemma in mcl2[0] if lemma]



print(mcl195)
print(mcl295)

print(sentences2[0])

print(unigram_similarity_mine(mcl195, mcl295, True))
print(unigram_similarity_mine(mcl195, mcl295, False))

[Lemma('leader.n.01.leader'), Lemma('give.v.01.give'), Lemma('new.a.01.new'), Lemma('opportunity.n.01.chance'), Lemma('let.v.01.let'), Lemma('seize.v.01.seize')]
[Lemma('leader.n.01.leader'), Lemma('profit.v.01.benefit'), Lemma('new.a.01.new'), Lemma('fortune.n.04.fortune'), Lemma('lashkar-e-taiba.n.01.Lashkar-e-Taiba'), Lemma('let.v.01.let'), Lemma('therefore.r.01.therefore'), Lemma('seize.v.01.seize')]
The leaders benefit aujourd' hui of a new luck and let's let them therefore seize it.
[Lemma('new.a.01.new'), Lemma('give.v.01.give'), Lemma('let.v.01.let'), Lemma('leader.n.01.leader'), Lemma('seize.v.01.seize'), Lemma('opportunity.n.01.chance')]
[Lemma('new.a.01.new'), Lemma('therefore.r.01.therefore'), Lemma('let.v.01.let'), Lemma('leader.n.01.leader'), Lemma('seize.v.01.seize'), Lemma('profit.v.01.benefit'), Lemma('lashkar-e-taiba.n.01.Lashkar-e-Taiba'), Lemma('fortune.n.04.fortune')]
0.5714285714285714
0.5714285714285714


In [ ]:

# conclusion: higher similarity scores with duplicates not removed

counter = 0
counter_eq = 0
for pair in zip(tp1, tp2):
  no_dups = unigram_similarity_mine(pair[0], pair[1], True)
  w_dups = unigram_similarity_mine(pair[0], pair[1], False)
  if no_dups > w_dups:
    counter +=1
  elif no_dups == w_dups:
    counter_eq +=1

print(counter)
print(counter_eq)
print(len(tp1) - counter - counter_eq)



['new', 'give', 'let', 'leader', 'seize', 'hope', 'chance']
['new', 'hui', 'luck', 'therefore', 'let', 'leader', 'aujourd', 'seize', 'benefit']
['change', 'amendment', 'propose', 'paragraph', 'reference', 'certain']
['change', 'amendment', 'propose', 'paragraph', 'reference', 'certain']
['fervent', 'let', 'include', 'supporter', 'remind', 'ally', 'tax']
['would', 'strong', 'like', 'remind', 'ally', 'tax', 'among']
['take', 'vote', 'today', 'place']
['take', 'vote', 'place']
['tire', 'inactive', 'disappointed', 'fisherman']
['tire', 'inactive', 'disappointed', 'fisherman']
['neither', 'qualified', 'revert', 'within', 'house', 'majority', 'article']
['voting', 'back', 'majority', 'article', 'parliament']
['small', 'country', 'increase', 'power', 'expense', 'big']
['baby', 'potency', 'country', 'augment', 'detriment', 'effect', 'big']
['tire', 'inactive', 'disappointed', 'fisherman']
['tire', 'inactive', 'disappointed', 'fisher']
['new', 'give', 'let', 'leader', 'seize', 'hope', 'chance']

In [ ]:
# the best option is to use synsets, as the most frequent occurrence is that all methods are equal
# but the second best option is synsets -- therefore, using synsets we will have the highest value
# (189 + 175) / 459 of the time for unigrams
# (266 + 116) / 459 of the time for bigrams
# (379 + 48) / 459 of the time for trigrams -- likely because there are fewer trigrams so 0 == 0

results = []
clean_counter = 0
lemma_counter = 0
syns_counter = 0
all_equal_ctr = 0
cln_lem_ctr = 0
lem_syns_ctr = 0
cln_syns_ctr = 0
for n,(c1, c2, m1, m2, s1, s2) in enumerate(zip(cln1, cln2, tp1, tp2, syn1, syn2)):
  print(n)
  clean = trigram_similarity(c1, c2, False)
  lemma = trigram_similarity(m1, m2, False)
  syns = trigram_similarity(s1, s2, False)
  if clean > lemma and clean > syns:
    clean_counter +=1
  elif lemma > clean and lemma > syns:
    lemma_counter += 1
  elif syns > lemma and syns > clean:
    syns_counter +=1
  elif clean == lemma == syns:
    all_equal_ctr += 1
  elif clean == lemma:
    cln_lem_ctr +=1
  elif lemma == syns:
    lem_syns_ctr += 1
  elif clean == syns:
    cln_syns_ctr += 1

  results.append(np.array([clean, lemma, syns]).argmax())

print(Counter(results))
print(clean_counter, lemma_counter, syns_counter)
print(all_equal_ctr, cln_lem_ctr, lem_syns_ctr, cln_syns_ctr)

sum([all_equal_ctr, cln_lem_ctr, lem_syns_ctr, cln_syns_ctr, clean_counter, lemma_counter, syns_counter])

0
[('leaders', 'given', 'new'), ('given', 'new', 'chance'), ('new', 'chance', 'let'), ('chance', 'let', 'hope'), ('let', 'hope', 'seize')]
[('leaders', 'benefit', 'aujourd'), ('benefit', 'aujourd', 'hui'), ('aujourd', 'hui', 'new'), ('hui', 'new', 'luck'), ('new', 'luck', 'lets'), ('luck', 'lets', 'let'), ('lets', 'let', 'therefore'), ('let', 'therefore', 'seize')]
0
[('leader', 'give', 'new'), ('give', 'new', 'chance'), ('new', 'chance', 'let'), ('chance', 'let', 'hope'), ('let', 'hope', 'seize')]
[('leader', 'benefit', 'aujourd'), ('benefit', 'aujourd', 'hui'), ('aujourd', 'hui', 'new'), ('hui', 'new', 'luck'), ('new', 'luck', 'let'), ('luck', 'let', 'let'), ('let', 'let', 'therefore'), ('let', 'therefore', 'seize')]
0
[(Synset('leader.n.01'), Synset('give.v.01'), Synset('new.a.01')), (Synset('give.v.01'), Synset('new.a.01'), Synset('opportunity.n.01')), (Synset('new.a.01'), Synset('opportunity.n.01'), Synset('let.v.01')), (Synset('opportunity.n.01'), Synset('let.v.01'), Synset('seiz

459

In [ ]:
clean = unigram_similarity_mine(cln1[90], cln2[90], False)
lemma = unigram_similarity_mine(tp1[90], tp2[90], False)
syns = unigram_similarity_mine(syn1[90], syn2[90], False)

In [ ]:
clean,lemma,syns

(1.0, 1.0, 1.0)

In [ ]:
10/13

0.7692307692307693

In [ ]:
9/16

0.5625

In [ ]:
6/13

0.46153846153846156

In [ ]:
def bigram_similarity(sentence1, sentence2, remove_duplicates):
  bigrams1 = list(nltk.bigrams(sentence1))
  bigrams2 = list(nltk.bigrams(sentence2))
  if remove_duplicates:
    bigrams1 = list(set(bigrams1))
    bigrams2 = list(set(bigrams2))
  total_bigrams = len(bigrams1) + len(bigrams2)
  used_bigrams = []
  count = 0
  for bigram in bigrams1:
    if bigram in bigrams2 and bigram not in used_bigrams:
      count += bigrams1.count(bigram)
      count += bigrams2.count(bigram)
    used_bigrams.append(bigram)
  print(bigrams1)
  print(bigrams2)
  print(total_bigrams)
  if total_bigrams == 0:
    return 0
  else:
    sim = count / total_bigrams
    return sim

In [ ]:
bigram_similarity(sent1, sent2, False)

[('increases', 'power'), ('power', 'big'), ('big', 'countries'), ('countries', 'expense'), ('expense', 'small'), ('small', 'countries'), ('countries', 'countries'), ('countries', 'countries')]
[('effect', 'increasing'), ('increasing', 'power'), ('power', 'large'), ('large', 'countries'), ('countries', 'expense'), ('expense', 'smaller'), ('smaller', 'countries')]
2


0.13333333333333333

In [ ]:
2/15

0.13333333333333333

In [ ]:
def trigram_similarity(sentence1, sentence2, remove_duplicates):
  trigrams1 = list(nltk.trigrams(sentence1))
  trigrams2 = list(nltk.trigrams(sentence2))
  if remove_duplicates:
    trigrams1 = list(set(trigrams1))
    trigrams2 = list(set(trigrams2))
  total_trigrams = len(trigrams1) + len(trigrams2)
  used_trigrams = []
  count = 0
  for trigram in trigrams1:
    if trigram in trigrams2 and trigram not in used_trigrams:
      count += trigrams1.count(trigram)
      count += trigrams2.count(trigram)
    used_trigrams.append(trigram)
  print(trigrams1)
  print(trigrams2)
  print(count)
  if total_trigrams == 0:
    return 0
  else:
    sim = count / total_trigrams
    return sim

In [ ]:
trigram_similarity(sent1, sent2, False)

[('increases', 'power', 'big'), ('power', 'big', 'countries'), ('big', 'countries', 'expense'), ('countries', 'expense', 'small'), ('expense', 'small', 'countries'), ('small', 'countries', 'countries'), ('countries', 'countries', 'countries')]
[('effect', 'increasing', 'power'), ('increasing', 'power', 'large'), ('power', 'large', 'countries'), ('large', 'countries', 'expense'), ('countries', 'expense', 'smaller'), ('expense', 'smaller', 'countries'), ('smaller', 'countries', 'countries'), ('countries', 'countries', 'countries')]
2


0.13333333333333333

In [ ]:
2/15

0.13333333333333333